In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('Project').getOrCreate()

In [0]:
df = spark.read.csv('/FileStore/tables/cruise_ship_info.csv',inferSchema=True,header=True)

In [0]:
df.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)

In [0]:
for item in df.head(1)[0]:
  print(item)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55

In [0]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_tranformed')
df_indexed = indexer.fit(df).transform(df)
df_indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------------------+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_tranformed|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------------------+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7| 1.0|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1| 1.0|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0| 1.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2| 1.0|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5| 1.0|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6| 1.0|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3| 1.0|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3| 1.0|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3| 1.0|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------------------+
only showing top 20 rows

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
df_indexed.columns

Out[13]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_tranformed']

In [0]:
assembler = VectorAssembler(inputCols=['Age',
                                       'Tonnage',
                                       'passengers',
                                       'length',
                                       'cabins',
                                       'passenger_density',
                                       'Cruise_line_tranformed']
                           ,outputCol='features')

In [0]:
output = assembler.transform(df_indexed)

In [0]:
output.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)
-- Cruise_line_tranformed: double (nullable = false)
-- features: vector (nullable = true)

In [0]:
df_analysis = output.select('features','crew')

In [0]:
train_data, test_data = df_analysis.randomSplit([0.7,0.3])

In [0]:
train_data.describe().show()

+-------+-----------------+
summary| crew|
+-------+-----------------+
 count| 107|
 mean|8.157383177570104|
 stddev| 3.47767416759258|
 min| 0.59|
 max| 21.0|
+-------+-----------------+

In [0]:
test_data.describe().show()

+-------+-----------------+
summary| crew|
+-------+-----------------+
 count| 51|
 mean|7.032156862745098|
 stddev|3.468045163330771|
 min| 0.59|
 max| 13.6|
+-------+-----------------+

In [0]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol='crew')

In [0]:
lr_model = lr.fit(train_data)

In [0]:
test_results = lr_model.evaluate(test_data)

In [0]:
test_results.residuals.show()

+--------------------+
 residuals|
+--------------------+
 -0.0819911463013625|
 -0.8510507508315603|
-0.20995861706140317|
 -0.630764166182944|
 0.7364327747381871|
 1.034667868370315|
-0.12043524980242815|
 1.7668742051013813|
 -0.3501447666088531|
-0.34807274730001936|
 -0.6023333736355241|
 1.5499068816714328|
 0.10857526371283654|
-0.34691293683741087|
 0.08483570431073673|
-0.01819174387232...|
-0.46765410822101927|
 -1.1344363873441132|
 -0.8014131224971539|
 0.1552840055368776|
+--------------------+
only showing top 20 rows

In [0]:
test_results.rootMeanSquaredError

Out[26]: 0.6148505084687664

In [0]:
test_results.r2

Out[27]: 0.9679395395217318

In [0]:
data_unlabeled = test_data.select('features')

In [0]:
predictions = lr_model.transform(data_unlabeled)

In [0]:
predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[5.0,133.5,39.59,...|13.211991146301363|
[6.0,30.276999999...| 4.40105075083156|
[7.0,158.0,43.7,1...|13.809958617061403|
[8.0,110.0,29.74,...|12.230764166182944|
[9.0,88.5,21.24,9...| 9.563567225261814|
[9.0,113.0,26.74,...|11.345332131629686|
[9.0,116.0,26.0,9...|11.120435249802428|
[10.0,46.0,7.0,6....|2.7031257948986185|
[10.0,58.825,15.6...| 7.350144766608853|
[10.0,81.76899999...| 8.76807274730002|
[10.0,110.0,29.74...|12.202333373635524|
[10.0,151.4,26.2,...|10.980093118328567|
[11.0,58.6,15.66,...| 7.491424736287163|
[11.0,90.09,25.01...| 8.826912936837411|
[12.0,2.329,0.94,...|0.5151642956892633|
[13.0,30.27699999...| 4.018191743872329|
[13.0,85.619,21.1...| 9.667654108221019|
[13.0,138.0,31.14...|12.894436387344113|
[14.0,77.104,20.0...| 8.801413122497154|
[14.0,83.0,17.5,9...| 9.294715994463122|
+--------------------+------------------+
only showing top 20 rows